<a href="https://colab.research.google.com/github/DanielBueno2001/Deep-Learning-Model/blob/main/Deep_Learning_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adagrad
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split

# Cargar el archivo CSV y verificar si existe
try:
    df = pd.read_csv('Conjunto_servidores_p_blicos_20240823.csv')
except FileNotFoundError:
    print("El archivo CSV no se encontró. Asegúrate de que el nombre y la ubicación sean correctos.")
    raise

# Eliminar las comas y convertir la columna "Asignación Básica Salarial" a formato numérico
df["Asignación Básica Salarial"] = df["Asignación Básica Salarial"].str.replace(",", "").astype(float)

# Mostrar las primeras filas después de la conversión
print(df["Asignación Básica Salarial"].head())

# Selección de columnas
columnas_a_mantener = ['Sexo', 'Meses de Experiencia Público', 'Meses de Experiencia Privado', "Nivel Educativo", "Departamento Ubicación Entidad", "Asignación Básica Salarial"]
df = df[columnas_a_mantener]

# Convertir columnas a numéricas
columnas_a_convertir = ["Meses de Experiencia Público", "Meses de Experiencia Privado", "Asignación Básica Salarial"]
df[columnas_a_convertir] = df[columnas_a_convertir].apply(pd.to_numeric, errors='coerce')

# Rellenar valores NaN con 0
df.fillna(0, inplace=True)

# Mapear valores categóricos a numéricos
df['Sexo'] = df['Sexo'].map({'Masculino': 0, 'Femenino': 1})
df['Departamento Ubicación Entidad'] = df['Departamento Ubicación Entidad'].map({
    'Amazonas': 1, 'Antioquia': 2, 'Arauca': 3, 'Atlántico': 4, 'Bolívar': 5, 'Boyacá': 6,
    'Caldas': 7, 'Caquetá': 8, 'Casanare': 9, 'Cauca': 10, 'Cesar': 11, 'Chocó': 12,
    'Córdoba': 13, 'Cundinamarca': 14, 'Guainía': 15, 'Guaviare': 16, 'Huila': 17,
    'La Guajira': 18, 'Magdalena': 19, 'Meta': 20, 'Nariño': 21, 'Norte de Santander': 22,
    'Putumayo': 23, 'Quindío': 24, 'Risaralda': 25, 'San Andrés y Providencia': 26,
    'Santander': 27, 'Sucre': 28, 'Tolima': 29, 'Valle del Cauca': 30, 'Vaupés': 31,
    'Vichada': 32
})

# Convertir "Nivel Educativo" a tipo string para evitar errores en la codificación
df['Nivel Educativo'] = df['Nivel Educativo'].astype(str)

# Codificación ordinal para "Nivel Educativo"
nivel_educativo_encoder = LabelEncoder()
df['Nivel Educativo'] = nivel_educativo_encoder.fit_transform(df['Nivel Educativo'])

# Separar características y etiqueta
X = df.drop(columns=['Asignación Básica Salarial'])
y = df["Asignación Básica Salarial"]

# División de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalar los datos
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Función para construir el modelo
def build_model(optimizer='adam', learning_rate=0.001, init='he_normal'):
    model = Sequential()
    model.add(Dense(64, activation='relu', kernel_initializer=init, input_shape=(X_train_scaled.shape[1],)))
    model.add(Dense(32, activation='relu', kernel_initializer=init))
    model.add(Dense(1))  # Salida para regresión

    model.compile(optimizer=optimizer(learning_rate=learning_rate), loss='mean_squared_error')
    return model

# Definir los hiperparámetros a probar
optimizers = {'adam': Adam, 'sgd': SGD}
learning_rates = [0.001, 0.01]
initializers = ['he_normal', 'glorot_uniform']
batch_sizes = [16, 32]
epochs = [50, 100]

# Variables para almacenar el mejor modelo y los mejores resultados
best_loss = float('inf')
best_model = None
best_params = None
best_results_df = None

# Iterar sobre todas las combinaciones de hiperparámetros
for optimizer_name in optimizers.keys():
    for lr in learning_rates:
        for init in initializers:
            for batch_size in batch_sizes:
                for epoch in epochs:
                    print(f"Entrenando con {optimizer_name}, lr={lr}, init={init}, batch_size={batch_size}, epochs={epoch}")

                    optimizer = optimizers[optimizer_name]
                    model = build_model(optimizer=optimizer, learning_rate=lr, init=init)

                    history = model.fit(
                        X_train_scaled, y_train,
                        epochs=epoch,
                        batch_size=batch_size,
                        validation_split=0.2,
                        verbose=1
                    )

                    # Evaluar el modelo en el conjunto de prueba
                    loss = model.evaluate(X_test_scaled, y_test, verbose=0)
                    print(f"Pérdida en el conjunto de prueba: {loss}")

                    # Calcular el error absoluto medio
                    y_pred = model.predict(X_test_scaled)
                    abs_error = np.abs(y_test - y_pred.flatten())
                    print(f"Error absoluto medio en el conjunto de prueba: {np.mean(abs_error)}")

                    # Guardar el mejor modelo
                    if loss < best_loss:
                        best_loss = loss
                        best_model = model
                        best_params = (optimizer_name, lr, init, batch_size, epoch)
                        best_results_df = pd.DataFrame({
                            'Valor Real': y_test.values,
                            'Predicción': y_pred.flatten(),
                            'Error Absoluto': abs_error
                        })

# Guardar los resultados en un archivo CSV si se encuentra el mejor modelo
if best_results_df is not None:
    best_results_df.to_csv('resultados_predicciones.csv', index=False)
    print("Los resultados se han guardado en 'resultados_predicciones.csv'.")
else:
    print("No se encontró ninguna configuración óptima durante la ejecución.")


<ipython-input-7-bf3c3323dbbf>:11: DtypeWarning: Columns (8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Conjunto_servidores_p_blicos_20240823.csv')


0     935000.0
1    1463133.0
2    5458553.0
3    1659503.0
4    1012471.0
Name: Asignación Básica Salarial, dtype: float64


/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1050: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1055: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1075: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Entrenando con adam, lr=0.001, init=he_normal, batch_size=16, epochs=50
Epoch 1/50
8153/8153 ━━━━━━━━━━━━━━━━━━━━ 26s 3ms/step - loss: nan - val_loss: nan
Epoch 2/50
8153/8153 ━━━━━━━━━━━━━━━━━━━━ 37s 3ms/step - loss: nan - val_loss: nan
Epoch 3/50
8153/8153 ━━━━━━━━━━━━━━━━━━━━ 18s 2ms/step - loss: nan - val_loss: nan
Epoch 4/50
8153/8153 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step - loss: nan - val_loss: nan
Epoch 5/50
8153/8153 ━━━━━━━━━━━━━━━━━━━━ 22s 2ms/step - loss: nan - val_loss: nan
Epoch 6/50
8153/8153 ━━━━━━━━━━━━━━━━━━━━ 18s 2ms/step - loss: nan - val_loss: nan
Epoch 7/50
8153/8153 ━━━━━━━━━━━━━━━━━━━━ 20s 2ms/step - loss: nan - val_loss: nan
Epoch 8/50
8153/8153 ━━━━━━━━━━━━━━━━━━━━ 20s 2ms/step - loss: nan - val_loss: nan
Epoch 9/50
8153/8153 ━━━━━━━━━━━━━━━━━━━━ 17s 2ms/step - loss: nan - val_loss: nan
Epoch 10/50
8153/8153 ━━━━━━━━━━━━━━━━━━━━ 18s 2ms/step - loss: nan - val_loss: nan
Epoch 11/50
8153/8153 ━━━━━━━━━━━━━━━━━━━━ 20s 2ms/step - loss: nan - val_loss: nan
Epoch 12/50
8

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

8153/8153 ━━━━━━━━━━━━━━━━━━━━ 19s 2ms/step - loss: nan - val_loss: nan
Epoch 2/100
8153/8153 ━━━━━━━━━━━━━━━━━━━━ 17s 2ms/step - loss: nan - val_loss: nan
Epoch 3/100
8153/8153 ━━━━━━━━━━━━━━━━━━━━ 18s 2ms/step - loss: nan - val_loss: nan
Epoch 4/100
8153/8153 ━━━━━━━━━━━━━━━━━━━━ 18s 2ms/step - loss: nan - val_loss: nan
Epoch 5/100
8153/8153 ━━━━━━━━━━━━━━━━━━━━ 23s 2ms/step - loss: nan - val_loss: nan
Epoch 6/100
8153/8153 ━━━━━━━━━━━━━━━━━━━━ 18s 2ms/step - loss: nan - val_loss: nan
Epoch 7/100
8153/8153 ━━━━━━━━━━━━━━━━━━━━ 17s 2ms/step - loss: nan - val_loss: nan
Epoch 8/100
8153/8153 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step - loss: nan - val_loss: nan
Epoch 9/100
8153/8153 ━━━━━━━━━━━━━━━━━━━━ 20s 2ms/step - loss: nan - val_loss: nan
Epoch 10/100
8153/8153 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step - loss: nan - val_loss: nan
Epoch 11/100
8153/8153 ━━━━━━━━━━━━━━━━━━━━ 17s 2ms/step - loss: nan - val_loss: nan
Epoch 12/100
8153/8153 ━━━━━━━━━━━━━━━━━━━━ 20s 2ms/step - loss: nan - val_loss: nan
E

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


4077/4077 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - loss: nan - val_loss: nan
Epoch 2/50
4077/4077 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - loss: nan - val_loss: nan
Epoch 3/50
4077/4077 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - loss: nan - val_loss: nan
Epoch 4/50
4077/4077 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - loss: nan - val_loss: nan
Epoch 5/50
4077/4077 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - loss: nan - val_loss: nan
Epoch 6/50
4077/4077 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - loss: nan - val_loss: nan
Epoch 7/50
4077/4077 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - loss: nan - val_loss: nan
Epoch 8/50
4077/4077 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - loss: nan - val_loss: nan
Epoch 9/50
4077/4077 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - loss: nan - val_loss: nan
Epoch 10/50
4077/4077 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - loss: nan - val_loss: nan
Epoch 11/50
4077/4077 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step - loss: nan - val_loss: nan
Epoch 12/50
4077/4077 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - loss: nan - val_loss: nan
Epoch 13/50
4077/

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


4077/4077 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - loss: nan - val_loss: nan
Epoch 2/100
4077/4077 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - loss: nan - val_loss: nan
Epoch 3/100
4077/4077 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step - loss: nan - val_loss: nan
Epoch 4/100
4077/4077 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - loss: nan - val_loss: nan
Epoch 5/100
4077/4077 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - loss: nan - val_loss: nan
Epoch 6/100
4077/4077 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - loss: nan - val_loss: nan
Epoch 7/100
4077/4077 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - loss: nan - val_loss: nan
Epoch 8/100
4077/4077 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - loss: nan - val_loss: nan
Epoch 9/100
4077/4077 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - loss: nan - val_loss: nan
Epoch 10/100
4077/4077 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - loss: nan - val_loss: nan
Epoch 11/100
4077/4077 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - loss: nan - val_loss: nan
Epoch 12/100
4077/4077 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - loss: nan - val_loss: nan
Epoch 13